In [1]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

load('src/time.py', import_globals=True, reload=True)
load('src/block.py', import_globals=True, reload=True)
load('src/color.py', import_globals=True, reload=True)
load('src/drawing.py', import_globals=True, reload=True)
load('src/interval.py', import_globals=True, reload=True)

<module 'src/interval.py' from 'src/interval.py'>

# Generate boilings

In [2]:
import math
import pandas as pd

from utils_ak.numeric import custom_round

In [3]:
import sys
# sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')
sys.path.append(r'C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat')
from app.models import *
from app.models import Boiling as BoilingModel
from app.models import Termizator as TermizatorModel

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
sqlite_filepath = r"C:\Users\arsen\Desktop\code\git\2020.10-umalat\umalat\data.sqlite"
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [4]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

df = pd.DataFrame(request.items(), columns=['name', 'qty'])
termizator = session.query(Termizator).first()
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())
sku = df.iloc[0]['sku']
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [5]:
boiling = df.iloc[0]['boiling']
termizator.pouring_time = 30

In [12]:
def make_boiling(line, boiling):
    maker = BlockMaker(default_push_func=dummy_push)
    make = maker.make

    line = '1'
    block_num = 12

    with make('boiling', block_num=block_num, line=line):
        # todo: make boiling size
        packing_t_time = 0

        timings = []
        timings.append(boiling.pourings.pouring_time)
        timings.append(boiling.pourings.soldification_time)
        timings.append(boiling.pourings.cutting_time)
        timings.append(boiling.pourings.pouring_off_time)
        timings.append(boiling.pourings.extra_time)

        total_time_size = sum(timings)
        packing_t_time += total_time_size
        with make('pouring', y=cast_row(f'water_{line}'), time_size=total_time_size):
            with make(y=0):
                make('termizator', time_size=termizator.pouring_time)
                make('pouring_name', time_size=total_time_size - termizator.pouring_time, name='чильеджина  3,6 альче  8500кг')
            with make(y=1):
                make('pouring_and_fermenting', time_size=timings[0])
                make('soldification', time_size=timings[1])
                make('cutting', time_size=timings[2])
                make('pouring_off', time_size=timings[3])
                make('extra', time_size=timings[4])


        # todo: specify parameter
        make('drenator', size=cast_t('04:00'), visible=False)
        packing_t_time += cast_t('04:00')  * 5

        timings = []
        timings.append(boiling.meltings.serving_time)
        # todo: specify number of kg 
        # todo: fix factor 1.3
        # melting time
        timings.append(custom_round(850 / boiling.meltings.speed * 60 * 1.3, 5, rounding='ceil'))

        total_time_size = sum(timings)
        packing_t_time += timings[0]
        packing_t_time += boiling.meltings.first_cooling_time
        packing_t_time += boiling.meltings.second_cooling_time
        
        with make('melting', y=10, time_size=total_time_size):
            with make(y=0):
                make('melting_label', time_size=4 * 5)
                make('melting_name', time_size=total_time_size - 4 * 5, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('serving', time_size=timings[0])
                make('melting_process', time_size=timings[1], speed=900)
            with make(y=2):
                make(time_size=timings[0], visible=False)
                make('cooling1', time_size=boiling.meltings.first_cooling_time)
                make('cooling2', time_size=boiling.meltings.second_cooling_time)

        timings = []
        timings.append(custom_round(850 / boiling.meltings.speed * 60 * 1.3, 5, rounding='ceil'))
        total_time_size = sum(timings)

        with make('packing', t=packing_t_time / 5, time_size=total_time_size, push_func=add_push):
            with make(y=0):
                make('packing_label', time_size=3 * 5)
                make('packing_name', time_size=total_time_size - 3 * 5, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with make(y=1):
                make('packing_brand', time_size=total_time_size, name='безлактозная/претто')
            # todo: add configurations
    #         with make(y=2):
    #             make('configuration', size=3, visible=False)
    #             make('configuration', size=1)
    #             make('configuration', size=5, visible=False)
    #             make('configuration', size=1)
    
    
    return maker.root.children[0]

In [13]:
# ROWS = {'water_1': 6, 'water_2': 9, 'salt_1': 15, 'salt_2': 18, 'water_melting': 24, 'water_packing': 29, 'salt_melting': 33, 'salt_packing': 46}
ROWS = {'water_1': 1, 'water_2': 3, 'water_melting': 5, 'water_packing': 8}

def cast_row(obj):
    if isinstance(obj, int):
        return obj
    elif isinstance(obj, str):
        return ROWS[obj]
    else:
        raise Exception('Unknown object type')
        
def boiling_validator(parent, boiling):
    b1 = max(parent.children, key=lambda b: b.rel_props.get('t', 0))
    b2 = boiling
    
    b1.rel_props['props_mode'] = 'absolute'
    b2.rel_props['props_mode'] = 'absolute'
    
    validate_disjoint(b1['pouring'][0]['termizator'], b2['pouring'][0]['termizator'])
    validate_disjoint(b1['melting'], b2['melting'])
    validate_disjoint(b1['packing'], b2['packing'])
    
    if b1.props['line'] == b2.props['line']:
        validate_disjoint(b1['pouring'], b2['pouring'])
        
    b1.rel_props.pop('props_mode')
    b2.rel_props.pop('props_mode')
    
    

In [16]:
root = Block('root', t=cast_t('01:45'))
boiling_1 = make_boiling(1, boiling)
boiling_2 = make_boiling(2, boiling)
dummy_push(root, boiling_1)
# dummy_push(root, boiling_2, beg='last_beg', max_tries=200, validator=boiling_validator)
root;

In [18]:
style = {
    'pouring': {'beg_time': '01:00'},
    'termizator': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
    'melting': {'beg_time': '07:00', 'y': cast_row('water_melting')},
    'melting_label': {'text': '{block_num}'},
    'melting_name': {'text': '{name}'},
    'serving': {'color': 'orange', 'text': 'подача и вымешивание'},
    'melting_process': {'color': 'orange', 'text': 'плавление {speed} кг/ч'},
    'cooling1': {'text': 'охлаж1 <{size} * 5> мин'},
    'cooling2': {'text': 'охлаж2 <{size} * 5> мин'},
    'packing': {'beg_time': '07:00', 'y': cast_row('water_packing')},
    'packing_label': {'text': '{block_num}'},
    'packing_name': {'text': '{name}'},
    'packing_brand': {'color': '#f2dcdb', 'text': '{name}'},
    'configuration': {'color': 'red'}
}

# update styles
for b in root.iter():
    if b.rel_props['class'] in style:
        b.rel_props.update(style[b.rel_props['class']])
        
work_book, sheet = init_sheet()
# work_book, sheet = init_template_sheet()
root.rel_props['index_width'] = 4
draw(sheet, root)
work_book.save('output.xlsx')

termizator 1 (14,20]
pouring_name 1 (20,40]
pouring_and_fermenting 2 (14,22]
soldification 2 (22,28]
cutting 2 (28,35]
pouring_off 2 (35,38]
extra 2 (38,40]
melting_label 5 (16,20]
melting_name 5 (20,37]
serving 6 (16,22]
melting_process 6 (22,37]
cooling1 7 (22,27]
cooling2 7 (27,37]
packing_label 8 (37.0,40.0]
packing_name 8 (40.0,52.0]
packing_brand 9 (37.0,52.0]
